In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

# Load required files
if not os.path.exists("parrot.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/parrot.bmp --no-check-certificate
if not os.path.exists("clock.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/clock.bmp --no-check-certificate
if not os.path.exists("chessboard.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/chessboard.bmp --no-check-certificate
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/lena.bmp --no-check-certificate
if not os.path.exists("firetruck.jpg") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/firetruck.jpg --no-check-certificate


I = cv2.imread('parrot.bmp')           # Read image
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY) # Convert to RGB

I2 = cv2.imread('clock.bmp')           # Read image
I2= cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY) # Convert to RGB

I3 = cv2.imread('chessboard.bmp')           # Read image
I3 = cv2.cvtColor(I3, cv2.COLOR_BGR2GRAY) # Convert to RGB

# Display
plt.figure(figsize=(I.shape[0]/100,I.shape[1]/100), dpi=200)
plt.imshow(I2, cmap ="gray")
plt.xticks([]), plt.yticks([])  # Hides the graph ticks and x / y axis
# plt.show()

# print(I2.shape)

# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
# Zadanie zawiera implementacje bicubic interpolation znalezioną w internecie z racji braku pliku zawierająca macierz
# https://en.wikipedia.org/wiki/Bicubic_interpolation 


import numpy as np
import matplotlib.pyplot as plt
import cv2

def base_function(x, a=-0.5):
    Wx = 0
    if np.abs(x)<=1:
        Wx = (a+2)*(np.abs(x)**3) - (a+3)*x**2 + 1
    elif 1<=np.abs(x)<=2:
        Wx = a*(np.abs(x)**3) - 5*a*(np.abs(x)**2) + 8*a*np.abs(x) - 4*a
    return Wx

def padding(img):
    h, w = img.shape
#     print(img.shape)
    pad_image = np.zeros(((h+4), (w+4)))
    pad_image[2:h+2, 2:w+2] = img
    
    return pad_image

  
def bicubic(img, vertical_ratio, horizontal_ratio):
    X, Y = img.shape 

    img = padding(img)
    
    
    height_img_resized = int(vertical_ratio*X)
    width_img_resized = int(horizontal_ratio*Y)  
    new_img = np.zeros((height_img_resized, width_img_resized))
    
    a=-0.75

    for i in range(width_img_resized):
        for j in range(height_img_resized):    
                        
            piksel_width = i/horizontal_ratio + 2
            piksel_height= j/vertical_ratio + 2
            
            piksel_height_int = int(piksel_height)
            piksel_width_int = int(piksel_width)

            u = piksel_width - piksel_width_int
            v = piksel_height - piksel_height_int
            
            #########################################
            
            piksel_height_int_2 = piksel_height_int + 2
            piksel_height_int_1 = piksel_height_int + 1
            piksel_height_int_ = piksel_height_int 
            piksel_height_int__ = piksel_height_int - 1
                
            piksel_width_int_2 = piksel_width_int + 2
            piksel_width_int_1 = piksel_width_int + 1
            piksel_width_int_ = piksel_width_int 
            piksel_width_int__ = piksel_width_int -1
            
            #########################################
            
            a11=img[piksel_height_int__, piksel_width_int__]
            a12=img[piksel_height_int__, piksel_width_int_]
            a13=img[piksel_height_int__, piksel_width_int_1]
            a14=img[piksel_height_int__, piksel_width_int_2]
            
            a21=img[piksel_height_int_, piksel_width_int__]
            a22=img[piksel_height_int_, piksel_width_int_]
            a23=img[piksel_height_int_, piksel_width_int_1]
            a24=img[piksel_height_int_, piksel_width_int_2]
            
            a31=img[piksel_height_int_1, piksel_width_int__]
            a32=img[piksel_height_int_1, piksel_width_int_]
            a33=img[piksel_height_int_1, piksel_width_int_1]
            a34=img[piksel_height_int_1, piksel_width_int_2]

            a41=img[piksel_height_int_2, piksel_width_int__]
            a42=img[piksel_height_int_2, piksel_width_int_]
            a43=img[piksel_height_int_2, piksel_width_int_1]
            a44=img[piksel_height_int_2, piksel_width_int_2]
            
            #########################################
            
            A = np.matrix([[base_function(u+1, a)], [base_function(u, a)], [base_function(u-1, a)], [base_function(u-2, a)]])
            C = np.matrix([base_function(v+1, a), base_function(v, a), base_function(v-1, a), base_function(v-2, a)])
            B = np.matrix([[a11, a12, a13 ,a14],
                           [a21, a22, a23, a24],
                           [a31, a32, a33, a34],
                           [a41, a42, a43, a44]])
            
            new_img[j, i] = np.dot(np.dot(C, B), A)
            
    return new_img
plt.gray()
f, (ax1, ax2) = plt.subplots(1,2, figsize=(30,30))
plt.gray()
ax1.axis('off')
ax1.imshow(bicubic(I,1.5,1.5))
ax2.imshow(I)
            
# Porównanie liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.
# w przypadku bicubic jest ich zdecydowanie więcej ->sam fakt liczenia pochodnych, tak samo dostęp do pamięci (chociażby w przyadku obliczania macierzy A,B,C)